<a href="https://colab.research.google.com/github/asokraju/Rienforcement-Learning/blob/master/David_Silver_Lecture_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Lecture 6: Value Function Approximators

This Document contains the simulation of examples provided in [Lecture 6](https://youtu.be/UoPei5o4fps?list=PLqYmG7hTraZDM-OYHWgPebj2MfCFzFObQ) of Introduction to Rienforcement Learning by David Silver. [Visit](http://www0.cs.ucl.ac.uk/staff/d.silver/web/Teaching_files/FA.pdf) for slides.

Author: Krishna Chaitanya Kosaraju

email: kkrishnachaitanya89@gmail.com


We consider the cart pole system to implement the RL algorithms
1. We use cart pole system to achieve this.
2. The state vector contains [cart position, cart velocity, pole angle, pole velocity]
3. cart position $\in [-2.4, 2.4]$
4. cart velocity $\in (-\infty, \infty)$
5. pole angle $\in (-41.8^o, 41.8^o)$
5. pole angular velocity $\in (-\infty, \infty)$
6. There are two discrete actions 0-push left, 1-push right

**Key note**: Postive values implies we are on the right side (clock-wise). For example, if pole angle  = $10^o$, then pole makes an angle $10^o$ with resptive to vertical axis and is bent towards right. 

Episode Termination
1. Pole Angle is more than ±12°
2. Cart Position is more than ±2.4 (center of the cart reaches the edge of the display)
3. Episode length is greater than 200 steps.


Reward
1. Reward is 1 for every step taken, including the termination step


See [Git](https://github.com/openai/gym/wiki/CartPole-v0) for more information.

In [0]:
#The following lines of code are necessary to render the environment from gym in colab
!apt-get install python-opengl -y

!apt install xvfb -y

!pip install pyvirtualdisplay

!pip install piglet


Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-430
Use 'apt autoremove' to remove it.
Suggested packages:
  libgle3
The following NEW packages will be installed:
  python-opengl
0 upgraded, 1 newly installed, 0 to remove and 7 not upgraded.
Need to get 496 kB of archives.
After this operation, 5,416 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 python-opengl all 3.1.0+dfsg-1 [496 kB]
Fetched 496 kB in 1s (422 kB/s)
Selecting previously unselected package python-opengl.
(Reading database ... 135004 files and directories currently installed.)
Preparing to unpack .../python-opengl_3.1.0+dfsg-1_all.deb ...
Unpacking python-opengl (3.1.0+dfsg-1) ...
Setting up python-opengl (3.1.0+dfsg-1) ...
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The f

In [0]:
# testing the above lines of code
#env = gym.make('CartPole-v0')
#env.reset()
#img = plt.imshow(env.render('rgb_array')) # only call this once
#for _ in range(100):
#    img.set_data(env.render('rgb_array')) # just update the data
#    display.display(plt.gcf())
#    display.clear_output(wait=True)
#    action = env.action_space.sample()
#    env.step(action)

In [0]:

#from pyvirtualdisplay import Display
#Display().start()

import gym
#from IPython import display
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import random

import sklearn.pipeline
import sklearn.preprocessing
 

from sklearn.linear_model import SGDRegressor
from sklearn.kernel_approximation import RBFSampler
from collections import namedtuple
import itertools
import sys

# MC learning using function approximation

In this section we  use function approximation to learn the Value function by MC learning methodology.



In [0]:
env = gym.envs.make('CartPole-v0')
state = env.reset()
action = env.action_space.sample()
next_state, reward, done,_ = env.step(action)

print(state, action, next_state)

[ 0.03441958  0.01239863 -0.02242909  0.02644624] 1 [ 0.03466756  0.20783493 -0.02190017 -0.27322808]


In [0]:
# we make some simple policy, which says if the cart is falling towards right ($\theta>0$) push right $action =1$ and vice-versa.
def Some_policy(state):
  if state[1]>0:
    return 1
  else:
    return 0
# Next we find the value function corresponding to this policy using function approximation

def Returns(states, rewards, gamma = 0.5):
  temp = 0
  Gt = []
  for indx, reward in enumerate(reversed(rewards)):
    temp = reward + gamma* temp
    Gt.append(temp)
  return list(reversed(Gt))
 

# lets initialize the Value function parameter w
w = np.full((1,49),1)

episodes = 1000000
alpha = 0.0001
Val = []
for episode in range(episodes):
  done = False
  state = env.reset()
  rewards =[]
  states = []
  actions = []
  while not done:
    states.append(state)
    action = Some_policy(state)
    actions.append(action)
    next_state, r, done, _ = env.step(action)
    rewards.append(r)
    state = next_state
  #since we are doing monte carlo we let the episode complete and store all the state, actions and rewards
  Gt = Returns(states, rewards, gamma =0.99)
  val_epi = []
  for i, s in enumerate(states):
    temp = np.array([[1, s[0], s[1], np.cos(s[1]), np.sin(s[1]), s[2], s[3]]])
    # we take a polynomial feature of degree 2 
    phi = np.reshape(np.outer(temp,temp.T),(1,49))
    V = np.inner(phi, w)
    w = w + alpha*(Gt[i] - V)*phi
    val_epi.append(V)
  Val.append(val_epi)
  print(episode, Gt[0])


#MC control

In [0]:
def eps_greedy_action(state, w, n_iter):
  s = state
  if n_iter <=10000:
    return random.choice([0,1])
  else:
    pre_feat = np.array([[1, s[0], s[1], np.cos(s[1]), np.sin(s[1]), s[2], s[3], 0, 1]])
    phi = np.reshape(np.outer(pre_feat,pre_feat.T),(1,81))
    Q_0 = np.inner(phi,w)

    pre_feat = np.array([[1, s[0], s[1], np.cos(s[1]), np.sin(s[1]), s[2], s[3], 1, 0]])
    phi = np.reshape(np.outer(pre_feat,pre_feat.T),(1,81))
    Q_1 = np.inner(phi,w)
    if Q_1 >= Q_0:
      temp = 1
    else:
      temp = 0
  temp_random = random.choice([0,1])
  return np.random.choice([temp, temp_random], size = 1, p = [1-(1/n_iter), 1/n_iter])[0]
  
# Next we find the value function corresponding to this policy using function approximation

def Returns(states, rewards, gamma = 0.99):
  temp = 0
  Gt = []
  for indx, reward in enumerate(reversed(rewards)):
    temp = reward + gamma* temp
    Gt.append(temp)
  return list(reversed(Gt))
 

# lets initialize the Value function parameter w
w = np.full((1,81),1)

episodes = 1000000
alpha = 0.01
Val = []
for episode in range(episodes):
  done = False
  state = env.reset()
  rewards =[]
  states = []
  actions = []
  while not done:
    states.append(state)
    action = eps_greedy_action(state, w, episode)
    actions.append(action)
    next_state, r, done, _ = env.step(action)
    rewards.append(r)
    state = next_state
  #since we are doing monte carlo we let the episode complete and store all the state, actions and rewards
  Gt = Returns(states, rewards, gamma =1)
  val_epi = []
  for i, s in enumerate(states):
    temp = np.array([[1, s[0], s[1], np.cos(s[1]), np.sin(s[1]), s[2], s[3], actions[i], 1-actions[i]]])
    # we take a polynomial feature of degree 2 
    phi = np.reshape(np.outer(temp,temp.T),(1,81))
    Q = np.inner(phi, w)
    w = w + alpha*(Gt[i] - Q)*phi
  if episode % 100 == 0:
    print(episode, Gt[0])


# Q learning

I am using the ideas from [here](https://github.com/dalmia/David-Silver-Reinforcement-learning/blob/master/Week%206%20-%20Value%20Function%20Approximations/Q-Learning%20with%20Value%20Function%20Approximation.ipynb)

In [0]:
env.reset()
def Samples_gen():
  """generates iid samples"""
  env.reset()
  state,_,_, _ =env.step(random.choice([0,1]))
  return state
iid_samples = np.array([Samples_gen() for i in range(10000)]) #set of i.i.d samples
print(iid_samples)
scaler =  sklearn.preprocessing.StandardScaler() #Initalizing a standard scalar to center the mean and variance
scaler.fit(iid_samples)# we need to initally fit them using scalar, so that we can use the transform later

#Feature expansion using FeatureUnion and radial bassis functions
featurizer = sklearn.pipeline.FeatureUnion([
        ("rbf1", RBFSampler(gamma=5.0, n_components=100)),
        ("rbf2", RBFSampler(gamma=2.0, n_components=100)),
        ("rbf3", RBFSampler(gamma=1.0, n_components=100)),
        ("rbf4", RBFSampler(gamma=0.5, n_components=100))
        ])
featurizer.fit(scaler.transform(iid_samples))

[[ 0.0061451  -0.241174   -0.04075995  0.30688855]
 [-0.0466714   0.24019821 -0.03761836 -0.35221596]
 [ 0.00745521 -0.19769155 -0.00165993  0.30736256]
 ...
 [ 0.02889282 -0.2304157   0.04066565  0.3546425 ]
 [-0.02249099 -0.22459392 -0.03491727  0.29945373]
 [ 0.04228178  0.23781948  0.00803595 -0.27345585]]


FeatureUnion(n_jobs=None,
             transformer_list=[('rbf1',
                                RBFSampler(gamma=5.0, n_components=100,
                                           random_state=None)),
                               ('rbf2',
                                RBFSampler(gamma=2.0, n_components=100,
                                           random_state=None)),
                               ('rbf3',
                                RBFSampler(gamma=1.0, n_components=100,
                                           random_state=None)),
                               ('rbf4',
                                RBFSampler(gamma=0.5, n_components=100,
                                           random_state=None))],
             transformer_weights=None, verbose=False)

In [0]:
class Estimator():
  def __init__(self):
    """
    We build a Q value function for each action.
    As a model we use SGD (with constant learning rate) on each featurized sample
    Method - featurized_state -  will do the scaling and feature expansion
    Method - predict - will Q values for a given state and action
    Method - update - will update the parameters using partial_fit
    """
    self.models = []
    for _ in range(env.action_space.n):
      model = SGDRegressor(learning_rate= "constant")
      model.partial_fit([self.featurized_state(env.reset())], [0])
      model.partial_fit([self.featurized_state(env.reset())], [1])
      #Above we need to use partial_fit, as we will be fitting the model for every sample
      self.models.append(model)
  def featurized_state(self, state):
    scaled = scaler.transform([state])
    temp = features.transform(scaled)
    return temp[0]
  def predict(self, state, actions = None):
    if actions is not None:
      prediction = self.models[a].predict([self.featurized_state(state)])
      return prediction[0]
    else:
      predictions = np.array([self.models[i].predict([self.featurized_state(state)]) for i in range(env.action_space.n)])
      return predictions.reshape(-1)
  def update(self, state, action, output):
    self.models[action].partial_fit([self.featurized_state(state)], [output])

#testing
temp = Estimator()
print(temp.models)
temp.update([1,1,1,2], 1, 20) #partial fit for first Q function
temp.update([1,1,1,2], 0, 20) #parial fit for the second Q function
temp.predict(env.reset()) # predicting Q values for each action

[SGDRegressor(alpha=0.0001, average=False, early_stopping=False, epsilon=0.1,
             eta0=0.01, fit_intercept=True, l1_ratio=0.15,
             learning_rate='constant', loss='squared_loss', max_iter=1000,
             n_iter_no_change=5, penalty='l2', power_t=0.25, random_state=None,
             shuffle=True, tol=0.001, validation_fraction=0.1, verbose=0,
             warm_start=False), SGDRegressor(alpha=0.0001, average=False, early_stopping=False, epsilon=0.1,
             eta0=0.01, fit_intercept=True, l1_ratio=0.15,
             learning_rate='constant', loss='squared_loss', max_iter=1000,
             n_iter_no_change=5, penalty='l2', power_t=0.25, random_state=None,
             shuffle=True, tol=0.001, validation_fraction=0.1, verbose=0,
             warm_start=False)]


array([0.24633533, 0.24633533])

In [0]:
def make_eps_greedy_policy(estimator, eps, nA = env.action_space.n):
  """
  returns a policy function
  """
  def policy_fn(observation):
    A = np.ones(nA, dtype=float)*eps/nA # [eps/n ....n.... eps/n] 
    q_values = estimator.predict(observation)
    best_action = np.argmax(q_values)
    A[best_action] += (1.0-eps) 
    return A
  return policy_fn
#test
temp_estimator = Estimator()
temp_polfun = make_eps_greedy_policy(estimator=temp_estimator, eps = 0.1, nA = 2)
temp_polfun([1,1,1,1])

array([0.95, 0.05])

In [0]:

def q_learning(env, estimator, num_episodes, discount_factor = 0.9, eps = 0.1, eps_decay = 0.9999):
  nA = env.action_space.n
  # keep track of stats
  EpisodeStats = namedtuple("Statistics", ["episode_lengths", "episode_rewards"])
  stats = EpisodeStats(episode_lengths=np.zeros(num_episodes),
                      episode_rewards=np.zeros(num_episodes)
                      )
  last_avg_reward = 0
  for i_epi in range(num_episodes):
    if i_epi>=200:
      last_avg_reward = np.round(np.mean(stats.episode_rewards[i_epi-100:i_epi - 1]),2)

    state = env.reset()
    policy = make_eps_greedy_policy(estimator, eps * eps_decay**i_epi, nA)
    last_reward = stats.episode_rewards[i_epi - 1]
    if i_epi %100 == 0:
      print("\rAverage Episode {}/{} ({})".format(i_epi + 1, num_episodes, last_avg_reward))
    print("\rEpisode {}/{} ({})".format(i_epi + 1, num_episodes, last_reward))
    sys.stdout.flush()
    
    for t in itertools.count():
      action = np.random.choice(nA, p = policy(state))
      new_state, reward, done, _ = env.step(action)
      new_action = np.random.choice(nA, p = policy(new_state))
      
      best_next_q_value = np.max(estimator.predict(new_state))
      target = reward + discount_factor*best_next_q_value
      estimator.update(state, action, target)
      state = new_state
      stats.episode_lengths[i_epi] = t
      stats.episode_rewards[i_epi] += reward

      if done:
        break
  return stats

In [0]:
estimator = Estimator()

In [0]:
stats = q_learning(env, estimator, 100*1000, eps=1, eps_decay=0.9999)

Average Episode 1/100000 (0)
Episode 1/100000 (0.0)
Episode 2/100000 (18.0)
Episode 3/100000 (30.0)
Episode 4/100000 (27.0)
Episode 5/100000 (14.0)
Episode 6/100000 (28.0)
Episode 7/100000 (22.0)
Episode 8/100000 (14.0)
Episode 9/100000 (28.0)
Episode 10/100000 (11.0)
Episode 11/100000 (15.0)
Episode 12/100000 (88.0)
Episode 13/100000 (27.0)
Episode 14/100000 (38.0)
Episode 15/100000 (44.0)
Episode 16/100000 (14.0)
Episode 17/100000 (15.0)
Episode 18/100000 (17.0)
Episode 19/100000 (11.0)
Episode 20/100000 (50.0)
Episode 21/100000 (49.0)
Episode 22/100000 (10.0)
Episode 23/100000 (18.0)
Episode 24/100000 (17.0)
Episode 25/100000 (22.0)
Episode 26/100000 (22.0)
Episode 27/100000 (12.0)
Episode 28/100000 (13.0)
Episode 29/100000 (11.0)
Episode 30/100000 (13.0)
Episode 31/100000 (14.0)
Episode 32/100000 (37.0)
Episode 33/100000 (33.0)
Episode 34/100000 (48.0)
Episode 35/100000 (28.0)
Episode 36/100000 (47.0)
Episode 37/100000 (14.0)
Episode 38/100000 (32.0)
Episode 39/100000 (10.0)
Episod

In [0]:
stats

Statistics(episode_lengths=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]), episode_rewards=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]))

In [0]:
stats.episode_lengths

array([ 0., 10.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [0]:
stats.episode_lengths[1] = 10

In [0]:
n